In [1]:
import polars as pl
import plotly.express as px

In [31]:
df = pl.read_parquet("f.parquet", columns=["frame", "preds"])
df.head()

frame,preds
i64,i64
1,0
2,0
3,0
4,0
5,0


In [49]:
df["frame"].max() // 3 / (60*60)

6.372222222222222

In [66]:
df["frame"].max() // (30*3)

764

In [74]:
!pip install -U polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 10.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: polars
    Found existing installation: polars 0.18.15
    Uninstalling polars-0.18.15:
      Successfully uninstalled polars-0.18.15


In [84]:
import plotly.express as px
from datetime import time
df_g = df.groupby(pl.col("frame") // (30*3)).agg((pl.sum("preds") > 3).cast(pl.Int8))
seconds = pl.col("frame")
df_g = (df_g
        .with_columns(pl.col("frame") * 30)
        .with_columns(hour=seconds // (60*60), minute=(seconds // 60) % 60, second=seconds % 60)
        .with_columns(timestamp=pl.time(pl.col("hour"), "minute", "second"))
        .sort("timestamp"))
print("% highlights", df_g["preds"].sum() / len(df_g))
print(df_g["timestamp"].max(), df_g["hour"].mean())
print(df_g.select(hour).max())

px.line(df_g, x="timestamp", y="preds", line_shape='hv')

% highlights 0.26143790849673204
06:22:00 2.7058823529411766
shape: (1, 1)
┌───────┐
│ frame │
│ ---   │
│ i64   │
╞═══════╡
│ 6     │
└───────┘


In [27]:
from datetime import date
dt = pl.col("timestamps").dt
df = df.with_columns(pl.datetime(year=pl.lit(2023), month=pl.lit(1), day=pl.lit(1), hour=dt.hour(), minute=dt.minute(), second=dt.second()))

In [29]:
px.scatter(df, x="datetime", y=pl.repeat(1, len(df), eager=True))